In [1]:
import numpy as np
import pandas as pd
import logging
import os
from datetime import timedelta
from pathlib import Path

from tinkoff.invest import CandleInterval, Client
from tinkoff.invest.utils import now

import sys
sys.path.append("..") 
from Portfolio.portfolio_tools import *
import tink_port as tink


/home/samsmu/anaconda3/envs/t/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/samsmu/anaconda3/envs/t/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


Вводим токен, получаем список доступных аккаунтов

In [2]:
#BASE = 't.UFRJ8SC9hafVOhFxEUY7yf1wZ1gGhwJp-WCp9o4rnEChHWns0c3jQ21eQwoOW_RurFqeZpss2scJkmMQnomJ9g'
BASE = 't.b1b-IVW3njbcDYPCPrrqAUMJdAD5gn5en82Jj8kb9uFkYOJmvrn3mF24By8QsHPvXz0zVC4_sqULvmG6yjUKww'
ETF = 't.6nHltT1dYSfrVTIV9zF72fxDlB2sXJbRD6iJNpZXTFAN61rmD7m71xPp9ko12ta1JxA06em4YdN36xicnBmjWg'
MOMENTUM = 't.24WV5_MMG1bQArK1WPp1_DYWD52f-VfGjpR1ci5Pqf0PJ948zWhDstoO_6d4wXIhFTMVsVJSgOzPElUIPEO4Mw'

token = BASE

In [3]:
accs = tink.get_accounts(token)
print("Количество аккаунтов:", len(accs.accounts))

print(accs.accounts[0].name)
account_id = accs.accounts[0].id

Количество аккаунтов: 1
Брокерский счёт


#### База идентификаторов Tinkoff

In [4]:
base = tink.get_id_base(token)
port = tink.get_portfolio(token)
df_port = tink.port_to_df(port, base)

In [37]:
df_port['sums'] = df_port.quantity * df_port.price
df_port

,figi,ticker,name,quantity,lot_quantity,price,sums
2,BBG004S68758,BANE,Башнефть,14,14,3157.00,44198.00
12,BBG00475K6C3,CHMF,Северсталь,28,28,1847.40,51727.20
6,BBG012YQ6P43,CIAN,Циан АДР,81,81,874.40,70826.40
0,BBG004731032,LKOH,ЛУКОЙЛ,1,1,7730.50,7730.50
4,BBG004S681B4,NLMK,НЛМК,140,14,220.42,30858.80
5,BBG00Y91R9T3,OZON,Ozon Holdings PLC,25,25,3806.00,95150.00
10,TCS00A103X66,POSI,Positive Technologies,33,33,2872.60,94795.80
1,BBG004S682Z6,RTKM,Ростелеком,1460,146,99.50,145270.00
11,BBG004730N88,SBER,Сбер Банк,150,15,306.28,45942.00
7,BBG004S681M2,SNGSP,Сургутнефтегаз - привилегированные акции,800,8,66.63,53304.00


### Portfolio Black-Litterman Model

In [8]:
mcaps = {}
for t in dfp.columns:
   # mcaps[t] = stock.info["marketCap"]
    mcaps[t] = 1
mcaps

{'LKOH': 1,
 'NLMK': 1,
 'AGRO': 1,
 'MAGN': 1,
 'TATN': 1,
 'SIBN': 1,
 'BANEP': 1,
 'SNGSP': 1,
 'POSI': 1,
 'SBER': 1,
 'CHMF': 1,
 'ROSN': 1}

In [9]:
mcaps['SBER'] = 1.5
mcaps['POSI'] = 1.5

In [28]:
imoex_figi = 'BBG333333333'
market_prices = tink.get_candles(token, imoex_figi, CandleInterval.CANDLE_INTERVAL_DAY)
market_prices =  tink.get_open_price(market_prices)

In [21]:
dfp

,LKOH,NLMK,AGRO,MAGN,TATN,SIBN,BANEP,SNGSP,POSI,SBER,CHMF,ROSN
date,,,,,,,,,,,,
2023-10-02,6716.5,210.50,1208.0,52.650,628.0,722.10,1468.0,58.500,2372.2,261.37,1371.0,538.50
2023-10-03,6720.0,209.96,1190.0,52.955,639.0,711.00,1439.5,56.800,2420.0,258.99,1392.0,534.65
2023-10-04,6740.0,208.86,1179.8,51.500,640.0,723.90,1432.5,56.650,2397.0,259.00,1355.6,538.20
2023-10-05,6706.0,212.84,1149.8,52.250,634.2,704.45,1475.0,55.475,2440.0,260.70,1370.0,536.00
2023-10-06,6749.5,210.80,1158.8,51.995,635.0,718.10,1460.0,57.090,2435.0,259.77,1369.4,534.50
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-21,7450.0,212.78,1411.8,54.800,695.7,755.70,2142.0,61.845,2674.0,295.99,1743.2,561.30
2024-03-22,7420.0,217.98,1422.6,55.520,704.1,756.80,2143.0,62.490,2680.0,295.77,1796.6,562.85
2024-03-25,7305.0,214.50,1388.0,54.800,694.5,737.00,2120.0,61.945,2642.2,294.00,1777.8,556.70


In [38]:
from pypfopt import black_litterman, risk_models
from pypfopt import BlackLittermanModel

S = risk_models.CovarianceShrinkage(dfp).ledoit_wolf()
delta = black_litterman.market_implied_risk_aversion(market_prices['ticker'])
delta

9.802403162030824

In [39]:
market_prior = black_litterman.market_implied_prior_returns(mcaps, delta, S)
market_prior

LKOH     0.235540
NLMK     0.250621
AGRO     0.404671
MAGN     0.222163
TATN     0.175982
SIBN     0.177426
BANEP    0.306354
SNGSP    0.265854
POSI     0.254061
SBER     0.201803
CHMF     0.241332
ROSN     0.239506
dtype: float64

In [41]:
# You don't have to provide views on all the assets
viewdict = {
    "POSI": 0.10,
}

confidences = [
    0.6
]
bl = BlackLittermanModel(S, pi=market_prior, absolute_views=viewdict)

In [43]:
intervals = [
    (0.1, 0.4),
]

variances = []
for lb, ub in intervals:
    sigma = (ub - lb)/2
    variances.append(sigma ** 2)

print(variances)
omega = np.diag(variances)

[0.022500000000000006]


In [44]:
# We are using the shortcut to automatically compute market-implied prior
bl = BlackLittermanModel(S, pi="market", market_caps=mcaps, risk_aversion=delta,
                        absolute_views=viewdict, omega=omega)

In [45]:
# Posterior estimate of returns
ret_bl = bl.bl_returns()
ret_bl

LKOH     0.210907
NLMK     0.225580
AGRO     0.377385
MAGN     0.198196
TATN     0.153239
SIBN     0.155086
BANEP    0.281036
SNGSP    0.241803
POSI     0.213216
SBER     0.178689
CHMF     0.218008
ROSN     0.216051
dtype: float64

In [46]:
rets_df = pd.DataFrame([market_prior, ret_bl, pd.Series(viewdict)], 
             index=["Prior", "Posterior", "Views"]).T
rets_df

,Prior,Posterior,Views
LKOH,0.235540,0.210907,NaN
NLMK,0.250621,0.225580,NaN
AGRO,0.404671,0.377385,NaN
MAGN,0.222163,0.198196,NaN
TATN,0.175982,0.153239,NaN
SIBN,0.177426,0.155086,NaN
BANEP,0.306354,0.281036,NaN
SNGSP,0.265854,0.241803,NaN
POSI,0.254061,0.213216,0.1
SBER,0.201803,0.178689,NaN


In [49]:
S_bl = bl.bl_cov()


In [50]:
from pypfopt import EfficientFrontier, objective_functions

ef = EfficientFrontier(ret_bl, S_bl)
ef.add_objective(objective_functions.L2_reg)
ef.max_sharpe()
weights = ef.clean_weights()
weights
     

D:\Soft\Anaconda\envs\t\Lib\site-packages\pypfopt\efficient_frontier\efficient_frontier.py:259: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  warnings.warn(


OrderedDict([('LKOH', 0.07897),
             ('NLMK', 0.08567),
             ('AGRO', 0.14173),
             ('MAGN', 0.07393),
             ('TATN', 0.05579),
             ('SIBN', 0.0562),
             ('BANEP', 0.10686),
             ('SNGSP', 0.09078),
             ('POSI', 0.08075),
             ('SBER', 0.06611),
             ('CHMF', 0.08197),
             ('ROSN', 0.08124)])

In [52]:
from pypfopt import DiscreteAllocation

da = DiscreteAllocation(weights, dfp.iloc[-1], total_portfolio_value=100000)
alloc, leftover = da.lp_portfolio()
print(f"Leftover: ${leftover:.2f}")
alloc

Leftover: $703.29


D:\Soft\Anaconda\envs\t\Lib\site-packages\cvxpy\problems\problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


{'LKOH': 1,
 'NLMK': 39,
 'AGRO': 10,
 'MAGN': 130,
 'TATN': 8,
 'SIBN': 8,
 'BANEP': 5,
 'SNGSP': 141,
 'POSI': 3,
 'SBER': 22,
 'CHMF': 4,
 'ROSN': 15}

### Model

## Load all shares

In [25]:
dfx = base[base["type"] == "shares"]
dfx = dfx[dfx["cur"] == "rub"]
base_ru = dfx.copy()


datetime.datetime(2024, 4, 1, 0, 0, tzinfo=datetime.timezone.utc)

In [28]:
res = []
for ind, pos in base_ru.iterrows():
#    print(pos.figi)        
    candles = tink.get_candles(token, pos.figi, CandleInterval.CANDLE_INTERVAL_DAY, now(),  50)
    df =  tink.get_open_price(candles)
    ticker = tink.figi_to_ticker(pos.figi, base)
    
    if ticker == None:
        ticker = pos.figi
    df.columns = [ticker]
    res.append(df)
    
df_full = pd.concat(res, axis = 1)

In [29]:
df_full

,KZIZP,IRKT,VSMO,UNAC,VKCO,TTLK,MGNT,SPBE,SVCB,KZIZ,...,LKOH,DELI,MGKL,LEAS,UDMN,NMTP,TGKA,CNTLP,MTLRP,MRKY
date,,,,,,,,,,,,,,,,,,,,,
2024-02-12,619.00,88.70,40260.0,1.5950,667.6,1.0890,7294.5,93.7,17.400,873.02,...,7240.0,272.85,2.3850,NaN,28500.00,12.500,0.009870,12.70,405.90,0.10500
2024-02-13,609.34,88.65,40020.0,1.5890,672.6,1.0800,7249.5,91.1,17.250,798.37,...,7297.0,270.30,2.3650,NaN,28999.91,12.575,0.009850,12.52,401.00,0.10690
2024-02-14,557.07,90.30,40220.0,1.5810,693.8,1.0870,7270.0,95.0,17.900,779.50,...,7275.0,266.95,2.3900,NaN,28899.93,12.800,0.010030,12.36,401.75,0.10645
2024-02-15,533.08,86.80,40280.0,1.5605,681.0,1.1030,7420.5,93.4,17.585,775.00,...,7205.0,282.00,2.3875,NaN,28468.06,12.645,0.009910,12.46,401.50,0.10475
2024-02-16,550.00,88.50,40100.0,1.5730,677.8,1.1255,7484.5,93.2,17.600,782.21,...,7294.0,281.00,2.3820,NaN,29000.00,13.005,0.009932,12.50,399.40,0.10290
2024-02-19,559.00,85.70,39260.0,1.5430,660.0,1.1080,7438.0,91.5,18.575,762.46,...,7233.0,280.15,2.3775,NaN,28595.97,12.890,0.009798,12.10,399.50,0.10210
2024-02-20,546.44,85.75,39340.0,1.5335,660.4,1.1010,7399.5,92.0,18.865,852.21,...,7188.0,287.75,2.4400,NaN,28550.76,13.250,0.009788,12.24,390.90,0.10170
2024-02-21,506.19,80.00,38200.0,1.5095,644.0,1.0680,7254.0,88.5,18.200,748.19,...,7080.0,282.00,2.3960,NaN,29998.94,12.475,0.009598,11.96,385.95,0.10105
2024-02-22,510.00,78.20,36840.0,1.5290,642.0,1.0000,7210.0,86.5,18.200,734.38,...,7000.5,274.05,2.3790,NaN,28980.00,12.020,0.009392,11.40,353.00,0.09795


#### Drops Momentum

In [11]:
dfp = df_full.copy()
drops = ["GTRK", "NTZL", "LSRG", 'TGKBP', 'FIVE', "VEON-RX"]
columns = [x for x in dfp.columns if x not in drops]
dfp = dfp[columns]


#### Drops Base

In [40]:
dfp = df_full.copy()
index_assets = pd.read_csv('index_assets.csv')['asset'].values.tolist()
columns = [x for x in dfp.columns if x in index_assets]

drops = ['FIVE']
columns = [x for x in columns if x not in drops]

dfp = dfp[columns]

### RiskFolio

In [42]:
import riskfolio as rp

rms= ['MV', 'MSV',  'CVaR']

lookback = 30
df_period = dfp[-lookback:]
df_period = df_period.dropna(axis = 1)

def riskfolio_weights(df_period, rm, obj, solvers):
    """
        rm  - MV, CVaR
        obj - Objective function, could be MinRisk, MaxRet, Utility or Sharpe
        solvers - ['MOSEK']
    """
    Y = df_period.pct_change().dropna()

    # Building the portfolio object
    port = rp.Portfolio(returns=Y)
    if solvers:
        port.solvers = solvers
    # Calculating optimum portfolio

    # Select method and estimate input parameters:

    method_mu='hist' # Method to estimate expected returns based on historical data.
    method_cov='hist' # Method to estimate covariance matrix based on historical data.

    port.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.94)

    # Estimate optimal portfolio:

    model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
    hist = True # Use historical scenarios for risk measures that depend on scenarios
    rf = 0 # Risk free rate
    l = 0 # Risk aversion factor, only useful when obj is 'Utility'
    # First we need to delete the cardinality constraint
    port.card = None 

    # Then we need to set the constraint on the minimum number of effective assets
    port.nea = 10
    w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)
    w = w[w.weights > 0.01]
    return w

#print_data(dfp)
df_period = dfp.dropna(axis = 1)

dfw = riskfolio_weights(df_period, 'CVaR', 'MaxRet', solvers = None)
dfw

,weights
WUSH,0.134600178
RTKM,0.128475661
SNGSP,0.064058175
AQUA,0.023789360
OZON,0.100876543
CHMF,0.073207537
BANE,0.057248987
CIAN,0.111449654
SBER,0.023608217
POSI,0.146552289


In [44]:
df_port['sums'] = df_port.quantity * df_port.price
sum_to_allocate = df_port.sums.sum() - 500
dfx = final_sums(dfw, sum_to_allocate, 50)
dfx['lot'] = 1
inds = dfx.index.values.tolist()

x = base_ru[base_ru['ticker'].isin (inds)]
s = x[['ticker', 'lot']].set_index('ticker')
dfx['lot'] = s

In [45]:
prices = dfp.iloc[-1].T.loc[dfx.index]
dfx["price"] = prices
dfx["lot_quantity"] = np.round(dfx.weights/ (dfx.price * dfx.lot)).astype(int)
dfx["quantity"] =  dfx.lot * dfx.lot_quantity
dfx["sum"]= dfx.price * dfx.lot_quantity * dfx.lot
dfx = dfx.sort_values("weights", ascending = False)
dfx.to_csv("t.csv")
dfx = dfx.reset_index()
dfx.columns = ['ticker', 'weights', 'lot', 'price', 'lot_quantity','quantity', 'sums']

dfx

,ticker,weights,lot,price,lot_quantity,quantity,sums
0,POSI,114650.0,1,2893.20,40,40,115728.00
1,WUSH,105300.0,1,323.22,326,326,105369.72
2,RTKM,100510.0,10,100.00,101,1010,101000.00
3,CIAN,87190.0,1,881.00,99,99,87219.00
4,OZON,78920.0,1,3786.00,21,21,79506.00
5,CHMF,57270.0,1,1862.20,31,31,57728.20
6,SNGSP,50110.0,100,66.35,8,800,53080.00
7,YNDX,49290.0,1,3931.00,13,13,51103.00
8,BANE,44790.0,1,3161.00,14,14,44254.00
9,NLMK,44200.0,10,221.58,20,200,44316.00


In [46]:
df_port

,figi,ticker,name,quantity,lot_quantity,price,sums
2,BBG004S68758,BANE,Башнефть,14,14,3157.00,44198.00
12,BBG00475K6C3,CHMF,Северсталь,28,28,1847.40,51727.20
6,BBG012YQ6P43,CIAN,Циан АДР,81,81,874.40,70826.40
0,BBG004731032,LKOH,ЛУКОЙЛ,1,1,7730.50,7730.50
4,BBG004S681B4,NLMK,НЛМК,140,14,220.42,30858.80
5,BBG00Y91R9T3,OZON,Ozon Holdings PLC,25,25,3806.00,95150.00
10,TCS00A103X66,POSI,Positive Technologies,33,33,2872.60,94795.80
1,BBG004S682Z6,RTKM,Ростелеком,1460,146,99.50,145270.00
11,BBG004730N88,SBER,Сбер Банк,150,15,306.28,45942.00
7,BBG004S681M2,SNGSP,Сургутнефтегаз - привилегированные акции,800,8,66.63,53304.00


In [48]:
def df_to_dict(dfx):
    return {row.ticker:row.lot_quantity for _,row in dfx.iterrows()}

old_port = df_to_dict(df_port)
new_port = df_to_dict(dfx)
new_port

{'POSI': 40,
 'WUSH': 326,
 'RTKM': 101,
 'CIAN': 99,
 'OZON': 21,
 'CHMF': 31,
 'SNGSP': 8,
 'YNDX': 13,
 'BANE': 14,
 'NLMK': 20,
 'AQUA': 20,
 'SBER': 6,
 'LKOH': 2}

In [49]:
old_port

{'BANE': 14,
 'CHMF': 28,
 'CIAN': 81,
 'LKOH': 1,
 'NLMK': 14,
 'OZON': 25,
 'POSI': 33,
 'RTKM': 146,
 'SBER': 15,
 'SNGSP': 8,
 'WUSH': 259,
 'YNDX': 15,
 None: 742}

In [50]:
def calculate_portfolio_difference(old_portfolio, new_portfolio):
    """
    Рассчитать разницу между двумя портфелями.

    Args:
        old_portfolio: Словарь, где ключом является тикер, а значением - количество акций.
        new_portfolio: Словарь, где ключом является тикер, а значением - количество акций.

    Returns:
        Словарь, где ключом является тикер, а значением - разница между количеством акций в 
        новых и старых портфелях.
    """

    difference = {}
    for ticker in new_portfolio:
        if ticker in old_portfolio:
            difference[ticker] = new_portfolio[ticker] - old_portfolio[ticker]
        else:
            difference[ticker] = new_portfolio[ticker]

    for ticker in old_portfolio:
        if ticker not in new_portfolio:
            difference[ticker] = -old_portfolio[ticker]
    # Сортировка по значению, по возрастанию
    sorted_diff = sorted(difference.items(), key=lambda x: x[1])

    return sorted_diff

In [51]:
rebalance = calculate_portfolio_difference(old_port, new_port)

flag_rebalance = True

In [53]:
for asset, qty in rebalance:
    if asset is None:
        continue
        
    print(asset, qty)

RTKM -45
SBER -9
OZON -4
YNDX -2
SNGSP 0
BANE 0
LKOH 1
CHMF 3
NLMK 6
POSI 7
CIAN 18
AQUA 20
WUSH 67


In [56]:
from tinkoff.invest import OrderDirection, OrderType
residuals = []
if flag_rebalance:
    with Client(token) as client:

        for asset, qty in rebalance:
            print(asset)
            if asset is None:
                continue
                
            figi = tink.ticker_to_figi(asset, base)
            trading_status = client.market_data.get_trading_status(
                figi=figi
            )
            
            if trading_status.market_order_available_flag and trading_status.api_trade_available_flag:
                if qty < 0:
                    resp = client.orders.post_order(figi=figi,
                                quantity= -qty,
                                direction=OrderDirection.ORDER_DIRECTION_SELL,
                                account_id=account_id,
                                order_type=OrderType.ORDER_TYPE_MARKET,)
                elif qty > 0:
                    resp = client.orders.post_order(figi=figi,
                        quantity=qty,
                        direction=OrderDirection.ORDER_DIRECTION_BUY,
                        account_id=account_id,
                        order_type=OrderType.ORDER_TYPE_MARKET,)
            else:
                print("Не доступно")
                residuals.append((asset, qty))
    flag_rebalance = False

None
RTKM


6681c85a561520414e73fd849fed89d3 PostOrder INVALID_ARGUMENT 30042


SBER


RequestError: (<StatusCode.INVALID_ARGUMENT: (3, 'invalid argument')>, '30042', Metadata(tracking_id='6681c85a561520414e73fd849fed89d3', ratelimit_limit='300, 300;w=60', ratelimit_remaining=298, ratelimit_reset=14, message='not enough assets for a margin trade'))

In [32]:
residuals

[]